In [11]:
"""
Multi‑Agent Proof‑of‑Concept using LangChain + LangGraph
=======================================================
• Router node (LLM) → decides which specialist agent should handle the user input.
• Specialist agents:
    1. Web‑Search Agent  – uses Serper.dev to retrieve live information.
    2. Generic Assistant – a normal conversational agent.

Designed for use in a Jupyter notebook *or* as a stand‑alone script.
Modular layout makes it easy to extend each sub‑agent with its own tools.
"""

from pathlib import Path
import os
from dotenv import load_dotenv

In [12]:
from typing import List, Dict, Any
import requests

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.agents import Tool, initialize_agent, AgentExecutor, AgentType

# LangGraph (for orchestrating the nodes)
from langgraph.graph import StateGraph

In [13]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
SERPER_BASE_URL = os.getenv("SERPER_BASE_URL", "https://google.serper.dev/")

In [14]:
llm_router = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY
)

llm_generic = ChatOpenAI(
    model="gpt-4o-mini", temperature=0.2, openai_api_key=OPENAI_API_KEY
)

In [15]:
# ───────────────────────────── Serper Search Tool ──────────────────────
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_community.tools.google_serper.tool import GoogleSerperRun
from langchain.tools import RequestsGetTool   

serper_wrapper = GoogleSerperAPIWrapper()            # uses SERPER_API_KEY
search_tool    = GoogleSerperRun(api_wrapper=serper_wrapper)
#read_url_tool  = RequestsGetTool()  

search_tools = [search_tool] 

In [16]:
# ─────────────────────────── Specialist Agents ─────────────────────────
SYSTEM_PROMPT_SEARCH = (
    "You are the **Web‑Search Agent**. Use the `serper_search` tool provided to look up "
    "information on the public internet. Cite the search findings when you answer.\n\n"
    "If the user's question cannot be answered with a web search, reply **only** with\n"
    "`SEARCH_AGENT_UNABLE`."
)

SYSTEM_PROMPT_GENERIC = (
    "You are the **Generic Assistant Agent**. Hold open‑ended conversations, explain\n"
    "concepts, write code – anything that does not need fresh web data.\n\n"
    "If the user explicitly asks for current events outside your knowledge, reply **only** with\n"
    "`GENERIC_AGENT_UNABLE`."
)

In [17]:
search_agent: AgentExecutor = initialize_agent(
    tools=search_tools,
    llm=llm_generic,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    system_message=SystemMessage(content=SYSTEM_PROMPT_SEARCH),
)

# ─── Generic conversational agent ────────────────────────────────────────────
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


## 5b. Generic conversational agent – ConversationChain (no functions array, no tools)
generic_memory = ConversationBufferMemory(memory_key="history", return_messages=True)
generic_agent = ConversationChain(
    llm=llm_generic,
    memory=generic_memory,
    verbose=False,
)

C:\Users\pierre.leroux\AppData\Local\Temp\ipykernel_12576\3635514508.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  search_agent: AgentExecutor = initialize_agent(
C:\Users\pierre.leroux\AppData\Local\Temp\ipykernel_12576\3635514508.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  generic_memory = ConversationBuf

In [18]:
# ─────────────────────────────── Router LLM ────────────────────────────
ROUTER_PROMPT = (
    "You are the **Router Agent**. Decide whether the user's message needs a live web\n"
    "search or can be handled by the generic assistant.\n\n"
    "Return exactly one word:**\n"
    "  • `SEARCH`  – if an up‑to‑date factual lookup is required\n"
    "  • `GENERIC` – for everything else\n\n"
    "Never output anything other than those two tokens."
)

def route(user_input: str) -> str:
    decision = llm_router([
        SystemMessage(content=ROUTER_PROMPT),
        HumanMessage(content=user_input),
    ]).content.strip().upper()
    return "SEARCH" if decision == "SEARCH" else "GENERIC"


In [19]:
# ────────────────────────────── LangGraph Flow ─────────────────────────
class AgentState(dict):
    """Mutable state passed between nodes."""
    messages: List[Dict[str, str]]  # running chat transcript
    last_user_msg: str             # the raw user input
    route: str | None                     # label chosen by router


def router_node(state: AgentState) -> AgentState:
    user_msg: str = state["last_user_msg"]
    route_label = route(user_msg) 
    print(f"🛣️  Router → {route_label}") 
    state["route"] = route(user_msg)
    return state


def search_agent_node(state: AgentState) -> AgentState:
    user_msg: str = state["last_user_msg"]
    answer = search_agent.run(user_msg)
    print("🔍 Search-agent finished") 
    state.setdefault("messages", []).append({"role": "assistant", "content": answer})
    return state


def generic_agent_node(state: AgentState) -> AgentState:
    user_msg: str = state["last_user_msg"]
    answer = generic_agent.run(user_msg)
    print("🔍 Generic-agent finished") 
    state.setdefault("messages", []).append({"role": "assistant", "content": answer})
    return state

# Build graph
sg = StateGraph(AgentState)
sg.add_node("router", router_node)
sg.add_node("search_agent", search_agent_node)
sg.add_node("generic_agent", generic_agent_node)

# Conditional edges
sg.add_conditional_edges("router", lambda s: s["route"], {
    "SEARCH": "search_agent",
    "GENERIC": "generic_agent",
})

sg.set_entry_point("router")

multi_agent_graph = sg.compile()

In [ ]:
# ───────────────────────────── Convenience API ─────────────────────────
def ask(message: str, state: AgentState | None = None) -> str:
    """Single‑shot helper (creates & returns a new state each call if omitted)."""
    state = state or {"messages": []}
    state["last_user_msg"] = message
    new_state = multi_agent_graph.invoke(state)
    return new_state["messages"][-1]["content"]

# ──────────────────────────────── ⏯ REPL ──────────────────────────────
if __name__ == "__main__":
    print("🤖  Multi‑Agent POC ready. Ask me anything!  (Press Ctrl‑C to quit)\n")
    state: AgentState = {"messages": []}
    try:
        while True:
            user_input = input("User: ")
            if not user_input.strip():
                continue
            state["last_user_msg"] = user_input
            state = multi_agent_graph.invoke(state)
            print("Assistant:", state["messages"][-1]["content"])
    except KeyboardInterrupt:
        print("\n👋  Bye!")

🤖  Multi‑Agent POC ready. Ask me anything!  (Press Ctrl‑C to quit)



C:\Users\pierre.leroux\AppData\Local\Temp\ipykernel_12576\3390523192.py:12: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  decision = llm_router([


🛣️  Router → SEARCH


C:\Users\pierre.leroux\AppData\Local\Temp\ipykernel_12576\4283351327.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = search_agent.run(user_msg)




> Entering new AgentExecutor chain...

Invoking: `google_serper` with `{'query': 'latest news on MrBeast'}`


MrBeast says game show allegations 'blown out of proportion'​​ Beast Games is set to stream next month amid allegations contestants were mistreated on set. MrBeast faces shock lawsuit from contestants on new Prime Video show. The Youtuber is being sued along with Amazon over allegations of a toxic work environment ... MrBeast Reveals He Lost Over $10 Million On Beast Games: "I Am An Idiot" ... MrBeast explained that he reinvested a lot of his income back into his brands and ... 75M followers · 710 following · 402 posts · @mrbeast: “New MrBeast or MrBeast Gaming video every single Saturday at noon eastern time!” MrBeast Top Stories. Musk, MrBeast or Shark Tank star: Who might buy TikTok after Trump's ultimatum? 21 Jan21st January Science & Tech. YouTube star MrBeast ... YouTuber MrBeast has disclosed a massive financial loss amounting to "tens of millions of dollars" from the 